# Identifying the Drivers of Churn at Telco
***
## Goals

My goal for this project is to create a model that will acurrately predict customer churn using the customer data provided. I will also be identifying the primary drivers of customer churn.

I will deliver classification_project.ipynb, README.md, predictions.csv, acquire.py, prepare.py, model.py, a walkthrough-style presentation of the project. The deliverables listed here are detailed in-depth in the README file.

***
## Acquire
Acquire data from the customers table from the telco_churn database on the codeup data science database server.

In [10]:
# preparing environment

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [218]:
# importing env for access to user, host and password
import acquire as ac

# display data frame
df = ac.get_telco_data()

df.head(3)

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,1,No,...,No,Yes,Yes,No,2,Yes,2,65.6,593.3,No
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,No,No,Yes,1,No,2,59.9,542.4,No
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,2,No,...,Yes,No,No,No,1,Yes,1,73.9,280.85,Yes


***
## Prepare
Prepare, tidy, and clean the data so it can be explored and analyzed

In [219]:
# displaying column names, dtypes, etc.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

- Categorical object variables such as gender, churn and dependents may need to be converted to 0s and 1s
- No missing values
- Need to rename tenure to reflect unit of measurement

In [220]:
# displaying various values for numerical columns (average, quartiles, min, max, etc)
# this allows us to get a rough idea of the values within the columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
senior_citizen,7043.0,0.162147,0.368612,0.00,0.0,0.00,0.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
internet_service_type_id,7043.0,1.872923,0.737796,1.00,1.0,2.00,2.00,3.00
contract_type_id,7043.0,1.690473,0.833755,1.00,1.0,1.00,2.00,3.00
payment_type_id,7043.0,2.315633,1.148907,1.00,1.0,2.00,3.00,4.00
monthly_charges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


- Senior_citizen: seniors are a minority
- Tenure: 72 month max means we have no records older than 6 years
- Type_ids for internet_service, contract, and paymenta are already numerical so no need to convert
- Monthly_charges: the average monthly payment is 64.76

In [221]:
# displaying unique values per column to identify categorical variables
obj_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
for col in obj_cols:
    if col != 'customer_id' and col != 'total_charges':
        print(df[col].value_counts())

Male      3555
Female    3488
Name: gender, dtype: int64
No     3641
Yes    3402
Name: partner, dtype: int64
No     4933
Yes    2110
Name: dependents, dtype: int64
Yes    6361
No      682
Name: phone_service, dtype: int64
No                  3390
Yes                 2971
No phone service     682
Name: multiple_lines, dtype: int64
No                     3498
Yes                    2019
No internet service    1526
Name: online_security, dtype: int64
No                     3088
Yes                    2429
No internet service    1526
Name: online_backup, dtype: int64
No                     3095
Yes                    2422
No internet service    1526
Name: device_protection, dtype: int64
No                     3473
Yes                    2044
No internet service    1526
Name: tech_support, dtype: int64
No                     2810
Yes                    2707
No internet service    1526
Name: streaming_tv, dtype: int64
No                     2785
Yes                    2732
No internet servic

- columns with 2 values may need to be converted to 0s and 1s
- columns with 3 or 4 values may need to be converted to numerical values

__Outliers and distributions__

In [222]:
# plotting data to identify outliers and distributions
"""for col in df.columns:
    if col != 'customer_id':
        plt.figure(figsize=(4,3))
        plt.hist(df[col])
        plt.title(col)
        plt.show()"""

"for col in df.columns:\n    if col != 'customer_id':\n        plt.figure(figsize=(4,3))\n        plt.hist(df[col])\n        plt.title(col)\n        plt.show()"

- No outliers found
- Customers without phone service and seniors are minorities
- Fairly even split between male and female
- Monthly_charges would be a somewhat normal distribution if not for the extremely high amount of low payment customers

__Prepare.py__

- convert object columns with binary values to 0s and 1s
- convert object columns with 3 - 4 unique values to numerical via dummy_df
- rename tenure to tenure_months
- remove duplicates
- split into train, validate, test

In [226]:
X = dfp.drop(['churn'], axis = 1)
y = dfp[['churn']]

X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size = .20, random_state = 123)

X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size = .30, random_state = 123)

print("X_train: ", X_train.shape, ", X_validate: ", X_validate.shape, ", X_test: ", X_test.shape)
print("y_train: ", y_train.shape, ", y_validate: ", y_validate.shape, ", y_test: ", y_test.shape)

X_train:  (3943, 34) , X_validate:  (1691, 34) , X_test:  (1409, 34)
y_train:  (3943, 1) , y_validate:  (1691, 1) , y_test:  (1409, 1)


In [239]:
def prepare_telco(x):
    x.drop_duplicates(inplace=True)
    cols = ["gender","partner","dependents","phone_service", "paperless_billing","churn"]
    x[cols] = x[cols].replace('Yes', 1)
    x[cols] = x[cols].replace('No', 0)
    dummy_df = pd.get_dummies(x[['multiple_lines','online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']])
    x = pd.concat([x, dummy_df], axis=1)
    cols_to_drop = ['multiple_lines','online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']
    x = x.drop(columns = cols_to_drop)
    x.rename(columns={'tenure':'monthly_tenure'}, inplace=True)
    x['total_charges'] = x.total_charges.where((x.monthly_tenure != 0),0)
    x['total_charges'] = x.total_charges.astype(float)
    return x
    
    

NameError: name 'prepare_telco' is not defined

In [1]:
from prepare import prepare_telco
import acquire as ac

prepare_telco(ac.get_telco_data())

,customer_id,gender,senior_citizen,partner,dependents,monthly_tenure,phone_service,internet_service_type_id,contract_type_id,paperless_billing,...,device_protection_Yes,tech_support_No,tech_support_No internet service,tech_support_Yes,streaming_tv_No,streaming_tv_No internet service,streaming_tv_Yes,streaming_movies_No,streaming_movies_No internet service,streaming_movies_Yes
0,0002-ORFBO,Female,0,1,1,9,1,1,2,1,...,0,0,0,1,0,0,1,1,0,0
1,0003-MKNFE,Male,0,0,0,9,1,1,1,0,...,0,1,0,0,1,0,0,0,0,1
2,0004-TLHLJ,Male,0,0,0,4,1,2,1,1,...,1,1,0,0,1,0,0,1,0,0
3,0011-IGKFF,Male,1,1,0,13,1,2,1,1,...,1,1,0,0,0,0,1,0,0,1
4,0013-EXCHZ,Female,1,1,0,3,1,2,1,1,...,0,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Female,0,0,0,13,1,1,2,0,...,0,0,0,1,1,0,0,1,0,0
7039,9992-RRAMN,Male,0,1,0,22,1,2,1,1,...,0,1,0,0,1,0,0,0,0,1
7040,9992-UJOEL,Male,0,0,0,2,1,1,1,1,...,0,1,0,0,1,0,0,1,0,0
7041,9993-LHIEB,Male,0,1,1,67,1,1,3,0,...,1,0,0,1,1,0,0,0,0,1
